# Homework Tasks

Read about difference between GPT-3.5 and GPT-4.

Read about metrics for generarive NLP.

**Advanced**: Generative models are usually very big. Read about model quantization. That may help with inference of big models such as GPT.

**Theory** (5 points): Google form questions.

**Practical task** (10 points):
1. Choose one:
    * Finetune transformer model for summarization on https://huggingface.co/datasets/samsum.
    * Finetune transformer model for translation on dataset of your choice.
2. Experiment with different prompts.
2. Based on a task you choose, choose a few metrics that are used in generative NLP (BLEU, ROUGE etc), test your finetune models using them, describe their pros and cons relative to the generations your model makes.

3. If you want, you can try use LoRA or prefix tuning for finetuning the model.

# Imports

In [1]:
!pip install transformers datasets evaluate peft py7zr rouge_score bert_score

import nltk
nltk.download('punkt')
import torch
import pandas as pd
import numpy as np
from evaluate import evaluator
import evaluate
from transformers import AutoTokenizer, GenerationConfig, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, AdamW
from datasets import load_dataset


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 27.5

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
class CFG:
    # model_path = 'facebook/bart-base'
    model_path = 't5-small'
    dataset_path = 'samsum'
    max_text_length = 512
    max_summary_length = 50

    # Training parameters
    fp16 = True
    learning_rate = 1e-4
    # weight_decay = 0.01
    warmup_steps = 50
    num_epochs = 3
    per_device_batch_size = 16



# Finetuning [Bart-base](https://huggingface.co/facebook/bart-base) model on [SAMSum Corpus ](https://huggingface.co/datasets/samsum)

# EDA

In [3]:
dataset = load_dataset(CFG.dataset_path)
print(dataset)

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})


SAMSun dataset already has training, validation and testing splits

Take a look at text structure in this dataset

In [4]:
for split in ['train', 'validation', 'test']:
    print(f'--------------------   {split.upper()} Split  --------------------')
    for i in range(1, 31, 10):
        print(dataset[split][i]['dialogue'])
        print(f"\n{dataset[split][i]['summary']}\n\n")

--------------------   TRAIN Split  --------------------
Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great

Olivia and Olivier are voting for liberals in this election. 


Mark: I just shipped the goods
Mark: Tomorrow I’ll send you the tracking number
George: Thanks!

Mark just shipped the goods and he will send George the tracking number tomorrow.


Aria: You won't believe who I've just met!
Aria: Charlie Evans!
Maverick: Oh God, I haven't seen him from ages!
Maverick: How is he doing?
Aria: He's doing great. :)
Aria: He got married, he runs a small family business, which he is very passionate about and generally he seems to be a happy and fulfilled man. :)
Aria: Oh, and he has two absolutely adorable daughters. :)
Aria: It was so nice to meet him, he's such a sweet soul.
Maverick: I’m glad to hear that. :)
Maverick: Time flies so fast, doesn't it?
Aria: It does. :) Recently I’ve met Cooper Roy, I'm sure you remember him, I co

Dataset has a dialogue and it's summary as training data

## Data processing

In [5]:
# Create tokenizer instance
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)

In [50]:
def add_prefix(examples):
    examples['dialogue'] = 'summarize: ' + examples['dialogue']
    return examples


def tokenize_dataset(examples):
    """
    Tokenizes dataset splits and return data ready for finetuning
    """
    model_inputs = tokenizer(examples['dialogue'], max_length=CFG.max_text_length, truncation=True)
    # separate max_length for summaries (labels)
    labels = tokenizer(examples['summary'], max_length=CFG.max_summary_length, truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [7]:
# Add 'summarize: ' prefix for T5 to function properly
dataset = dataset.map(add_prefix)

tokenized_dataset = dataset.map(tokenize_dataset, batched=True, remove_columns=['id', 'dialogue', 'summary'])

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [8]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})


Check encoded and decoded texts

In [9]:
for split in ['train', 'validation', 'test']:
    print(f'Tokenized text: {tokenized_dataset[split][0]["input_ids"]}')
    print(f'Decoded text: {tokenizer.decode(tokenized_dataset[split][0]["input_ids"], skip_special_tokens=True)}')

Tokenized text: [21603, 10, 21542, 10, 27, 13635, 5081, 5, 531, 25, 241, 128, 58, 16637, 10, 10625, 55, 21542, 10, 27, 31, 195, 830, 25, 5721, 3, 10, 18, 61, 1]
Decoded text: summarize: Amanda: I baked cookies. Do you want some? Jerry: Sure! Amanda: I'll bring you tomorrow :-)
Tokenized text: [21603, 10, 71, 10, 2018, 3059, 6, 33, 25, 3164, 5721, 22, 7, 3742, 58, 272, 10, 27, 22, 51, 1134, 417, 27, 183, 5, 363, 22, 7, 95, 58, 71, 10, 1072, 25, 281, 28, 140, 12, 8, 2586, 8596, 58, 5, 272, 10, 363, 103, 25, 241, 12, 103, 58, 71, 10, 27, 241, 12, 129, 3, 9, 17351, 21, 82, 520, 5, 272, 10, 466, 56, 143, 376, 78, 1095, 5, 71, 10, 11475, 6, 62, 22, 162, 5172, 34, 186, 648, 5, 27, 317, 3, 88, 22, 7, 1065, 230, 5, 272, 10, 466, 22, 7, 207, 5, 2922, 4890, 3, 9, 1782, 19, 3, 9, 3429, 962, 5, 2792, 578, 3, 9, 1871, 3, 117, 18, 61, 71, 10, 27, 31, 195, 129, 376, 80, 13, 273, 385, 3887, 5, 272, 10, 555, 24, 751, 31, 17, 1604, 95, 396, 600, 117, 18, 61, 71, 10, 275, 3, 1544, 396, 231, 117, 18, 61, 6

Create ROUGE metric and custom compute metrics function from [HuggingFace](https://huggingface.co/docs/transformers/tasks/summarization#evaluate)

In [10]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # print(predictions, labels)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True,  clean_up_tokenization_spaces=True)
    # print(decoded_preds, decoded_labels)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}


# def postprocess_text(preds, labels):
#     preds = [pred.strip() for pred in preds]
#     labels = [label.strip() for label in labels]

#     # rougeLSum expects newline after each sentence
#     preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
#     labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

#     return preds, labels


# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     # Replace -100s used for padding as we can't decode them
#     preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # Some simple post-processing
#     decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

#     result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     result = {k: round(v * 100, 4) for k, v in result.items()}
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     return result

Modeling

In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained(CFG.model_path)

optim = AdamW(model.parameters())

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    pad_to_multiple_of=8 if CFG.fp16 else None,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [13]:
# config to set model generation settings
generation_config = GenerationConfig(
    max_new_tokens = CFG.max_summary_length,
    num_beams = 4,
    early_stopping = True,
    decoder_start_token_id = model.config.decoder_start_token_id
)

train_args = Seq2SeqTrainingArguments(
    output_dir = './t5-small-summarization',
    per_device_train_batch_size = CFG.per_device_batch_size,
    per_device_eval_batch_size = CFG.per_device_batch_size,
    num_train_epochs = CFG.num_epochs,
    warmup_steps = CFG.warmup_steps,
    fp16=CFG.fp16,
    do_train = True,
    do_eval = True,
    evaluation_strategy = 'steps',
    eval_steps = 500,
    save_strategy = 'steps',
    save_total_limit=2,
    logging_strategy = 'steps',
    logging_steps = 100,
    dataloader_drop_last = True,
    # saves more VRAM if set to `False`
    dataloader_pin_memory = False,
    predict_with_generate = True,
    generation_config = generation_config
)

trainer = Seq2SeqTrainer(
    model = model,
    args = train_args,
    train_dataset = tokenized_dataset['train'],
    eval_dataset = tokenized_dataset['validation'],
    optimizers = (optim, None),
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics=compute_metrics
)


In [14]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,1.989400,1.834059,0.459700,0.221800,0.381500,0.381300,50.000000
1000,1.774300,1.754916,0.478900,0.236600,0.396600,0.395900,50.000000
1500,1.717700,1.708531,0.488400,0.242000,0.398100,0.397800,50.000000
2000,1.536800,1.697284,0.493100,0.244400,0.399400,0.399200,50.000000
2500,1.518700,1.670291,0.491900,0.242300,0.398200,0.397700,50.000000


TrainOutput(global_step=2760, training_loss=1.763051312211631, metrics={'train_runtime': 1055.4866, 'train_samples_per_second': 41.873, 'train_steps_per_second': 2.615, 'total_flos': 4556586265608192.0, 'train_loss': 1.763051312211631, 'epoch': 3.0})

# Prompt experiments

# Metrics results

BERTScore

In [52]:
# Instantiate the BERTScorer object for English language
# from bert_score import BERTScorer
# scorer = BERTScorer(lang="en")

# # Calculate BERTScore for the summary 1 against the excerpt
# # P1, R1, F1_1 represent Precision, Recall, and F1 Score respectively
# P1, R1, F1_1 = scorer.score([eval_summary_1], [ref_summary])

# # Calculate BERTScore for summary 2 against the excerpt
# # P2, R2, F2_2 represent Precision, Recall, and F1 Score respectively
# P2, R2, F2_2 = scorer.score([eval_summary_2], [ref_summary])

# print("Summary 1 F1 Score:", F1_1.tolist()[0])
# print("Summary 2 F1 Score:", F2_2.tolist()[0])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using evaluator from HuggingFace ecosystem

In [15]:
print(evaluate.list_evaluation_modules())

['precision', 'code_eval', 'roc_auc', 'cuad', 'xnli', 'rouge', 'pearsonr', 'mse', 'super_glue', 'comet', 'cer', 'sacrebleu', 'mahalanobis', 'wer', 'competition_math', 'f1', 'recall', 'mauve', 'xtreme_s', 'bleurt', 'ter', 'accuracy', 'exact_match', 'indic_glue', 'spearmanr', 'mae', 'squad', 'chrf', 'glue', 'perplexity', 'mean_iou', 'squad_v2', 'meteor', 'bleu', 'wiki_split', 'sari', 'google_bleu', 'bertscore', 'matthews_correlation', 'seqeval', 'trec_eval', 'rl_reliability', 'angelina-wang/directional_bias_amplification', 'cpllab/syntaxgym', 'kaggle/ai4code', 'codeparrot/apps_metric', 'mfumanelli/geometric_mean', 'poseval', 'brier_score', 'abidlabs/mean_iou', 'abidlabs/mean_iou2', 'giulio98/codebleu', 'mase', 'mape', 'smape', 'dvitel/codebleu', 'NCSOFT/harim_plus', 'JP-SystemsX/nDCG', 'Drunper/metrica_tesi', 'jpxkqx/peak_signal_to_noise_ratio', 'jpxkqx/signal_to_reconstruction_error', 'hpi-dhc/FairEval', 'nist_mt', 'charcut_mt', 'ybelkada/cocoevaluate', 'harshhpareek/bertscore', 'posicu

In [ ]:
summaries_evaluator = evaluator("summarization")
test_metrics = summaries_evaluator.compute(
    model_or_pipeline = '/content/t5-small-summarization/checkpoint-2500',
    data = dataset['test'],
    input_column = 'dialogue',
    label_column = 'summary',
    metric = evaluate.combine([
        # For some reason BERTScore cant load with required 'lang' parameter
        # evaluate.load('bertscore', lang='en', batch_size=CFG.per_device_batch_size, use_fast_tokenizer=True),
        evaluate.load("rouge"),
        evaluate.load("bleu")
    ])
)

In [27]:
print(test_metrics)

{'rouge1': 0.3289981785063752, 'rouge2': 0.1476699763409999, 'rougeL': 0.28229238059954925, 'rougeLsum': 0.28000689636879655, 'bleu': 0.08840272953686946, 'precisions': [0.31100478468899523, 0.12315270935960591, 0.050761421319796954, 0.031413612565445025], 'brevity_penalty': 1.0, 'length_ratio': 1.3483870967741935, 'translation_length': 209, 'reference_length': 155, 'total_time_in_seconds': 3.1426759290002337, 'samples_per_second': 1.9092009916239612, 'latency_in_seconds': 0.523779321500039}


# LoRA finetuning